In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# ! cat "/content/drive/My Drive/CS221/transformers-master/src/transformers/tokenization_auto.py"
# ! cat "/content/drive/My Drive/CS221/transformers-master/src/transformers/modeling_auto.py"
import sys
sys.path.append('/content/drive/My Drive/CS221/transformers-master/src')

In [ ]:
! pip install transformers
! pip install sacremoses
! pip install bert-extractive-summarizer

     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 23.7MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
     |████████████████████████████████| 2.9MB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a4353138693470d93b7259c5497a37980a9e3542ebff86d2fd25230b2360b11b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Data

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/bbc_data_original.csv')

In [ ]:
dataset.head()

,Unnamed: 0,File,Text,Summary
0,0,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
1,1,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
2,2,006.txt,Japan narrowly escapes recession\n\nJapan's ec...,"On an annual basis, the data suggests annual g..."
3,3,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...
4,4,007.txt,Jobs growth still slow in the US\n\nThe US cre...,The job gains mean that President Bush can cel...


In [ ]:
def remove_special_characters_and_to_lowercase(s):
    # remove special characters except A-Z, a-z, 0-9, and space.
    s = re.sub(r'[^\'A-Za-z0-9]+', ' ', s)
    # to lowercase
    return s.lower()

In [ ]:
labels = []
for i in range(dataset["Text"].size):
  raw_label = dataset["Summary"].get(i)
  labels.append(remove_special_characters_and_to_lowercase(raw_label))

Code

In [ ]:
from summarizer import TransformerSummarizer

In [ ]:
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")

Evaluation

In [ ]:
# Because the actual output (500~1000 words) is usually much longer than the
# predicted output, instead of how many n-grams are matched in the actual
# output, we compute the number of bigrams in the predicted output that also
# appear in the actual output.
#
# Params
# n: n-gram to compare.
def computeCustomizedRougeN(predicted_output: str, actual_output: str, n: int):
  predicted_words = predicted_output.split()
  actual_words = actual_output.split()
  assert n <= len(predicted_words) and n <= len(actual_words)
  predicted_ngrams = [''.join(predicted_words[i : i + n]) for i in range(len(predicted_words) - n + 1)]
  actual_ngrams = set([''.join(actual_words[i : i + n]) for i in range(len(actual_words) - n + 1)])
  num_matches = 0
  for predicted_ngram in predicted_ngrams:
    if predicted_ngram in actual_ngrams:
      num_matches += 1
  return num_matches / len(predicted_ngrams)

In [ ]:
computeCustomizedRougeN("I have apple", "I have an apple", 2)  # 0.5 since "I have" is a match, but "have apple" is not.

0.5

In [ ]:
dictionary = {'Text': [], 'Label': [], 'Summary': [], 'Rouge2': [], 'Rouge3': [], 'Rouge4': []}

for i in range(dataset["Text"].size):
  print('='*20, "Round", i, '='*20)
  input = dataset["Text"].get(i)
  label = labels[i]
  raw_output = ''.join(model(input, min_length=40, max_length=150))
  output = remove_special_characters_and_to_lowercase(raw_output)
  custom_rouge2_score = computeCustomizedRougeN(output, label, 2)
  custom_rouge3_score = computeCustomizedRougeN(output, label, 3)
  custom_rouge4_score = computeCustomizedRougeN(output, label, 4)
  dictionary['Text'].append(input)
  dictionary['Label'].append(label)
  dictionary['Summary'].append(output)
  dictionary['Rouge2'].append(custom_rouge2_score)
  dictionary['Rouge3'].append(custom_rouge3_score)
  dictionary['Rouge4'].append(custom_rouge4_score)

==================== Round 0 ====================
==================== Round 1 ====================
==================== Round 2 ====================
==================== Round 3 ====================
==================== Round 4 ====================
==================== Round 5 ====================
==================== Round 6 ====================
==================== Round 7 ====================
==================== Round 8 ====================
==================== Round 9 ====================
==================== Round 10 ====================
==================== Round 11 ====================
==================== Round 12 ====================
==================== Round 13 ====================
==================== Round 14 ====================
==================== Round 15 ====================
==================== Round 16 ====================
==================== Round 17 ====================
==================== Round 18 ====================
==================== Round 19 ===========

In [ ]:
print(sum(dictionary['Rouge2']) / len(dictionary['Rouge2']))
print(sum(dictionary['Rouge3']) / len(dictionary['Rouge3']))
print(sum(dictionary['Rouge4']) / len(dictionary['Rouge4']))

0.526507626250118
0.47988923810267203
0.45780922293608667
